# Finding Time-to-Targets
#### 10/29/23

One of the questions that Kai and I are interested in examining is whether or not listeners are quicker to respond correctly to modal voice than creaky voice. To do this, we'll need to collect the participants' reaction times. We already have the _response time_ (the duration between when the audio file starts playing and when the participant answers), but we're more interested in getting the response time after the participant hears the target word. To be specific:

`reaction_time` = `response_time` - `time_to_target`

Unfortunately, we don't have all the time-to-targets, so we'll have to manually fill in the gaps.

## Strategy

To get the time-to-targets of each audio file, we will:
1) Annotate the pre-target duration, and save the annotation in a TextGrid file. 
2) Save the durations of each annotation to a csv file.

The first step can't be automated, since finding the target is really hard and a bit of a subjective process. However, it can be done relatively quickly with some Praat code. The second should be much easier.

## TextGrid annotation

Maxine has a basic TextGrid annotator script that we can use. Here's basically what it does:
- Prompt the user for a directory with .wav files
- For each .wav file,
    - Prepare a .TextGrid file with the same basename (unless it already exists; skip if so)
    - Let the user view and edit the file
    - Prompt the user to create an annotated boundary
    - Save the annotation to the TextGrid file

In [1]:
praat_script = """
procedure getTextGrid: .wav$
    .path$ = replace$: .wav$, ".wav", ".TextGrid", 0

    if !fileReadable: .path$
        .textgrid = To TextGrid: tiers$, point_tiers$

    elif if_TextGrid_already_exists == 2
        .textgrid = To TextGrid: tiers$, point_tiers$
        .default$ = mid$: .path$, rindex (.path$, "/") + 1, length (.path$)
        .default$ = replace$: .default$, ".wav", ".TextGrid", 1

        .path$ = chooseWriteFile$: "TextGrid already exists in folder. 
        ... Choose where to save the new TextGrid.", .default$

    elif if_TextGrid_already_exists == 3
        .textgrid = Read from file: .path$

    endif

endproc

procedure getFiles: .dir$, .ext$
    .obj = Create Strings as file list: "files", .dir$ + "/*" + .ext$
    .length = Get number of strings

    for .i to .length
        .fname$ = Get string: .i
        .files$ [.i] = .dir$ + "/" + .fname$

    endfor

    removeObject: .obj

endproc

## MAIN
directory$ = chooseDirectory$: "Choose a directory with wav files to annotate."

form Arguments
    sentence Interval_tiers vowel
    sentence Point_tiers 
    optionmenu If_TextGrid_already_exists: 1
        option skip the wav file
        option create a new TextGrid
        option open the existing TextGrid
    comment Press OK to choose the directory of wav files to annotate.
endform

tiers$ = interval_tiers$ + " " + point_tiers$
@getFiles: directory$, ".wav"

for i to getFiles.length
    wav = Read from file: getFiles.files$ [i]

    @getTextGrid: getFiles.files$ [i]

    if !fileReadable (getTextGrid.path$) or if_TextGrid_already_exists > 1
        selectObject: wav, getTextGrid.textgrid

        View & Edit

        beginPause: "Annotation"
            comment: "Press OK when done to save."

        endPause: "OK", 0

        selectObject: getTextGrid.textgrid
        Save as text file: getTextGrid.path$

        removeObject: getTextGrid.textgrid

    endif

    removeObject: wav
endfor
"""

There are two things I want to do differently. Firstly, I want to save the TextGrids in a separate folder. This will make it much easier to loop through all of them later. Secondly, I want to print out the name of the annotation so I can copy and paste instead of typing (typing is tedious and easy to get wrong):

In [2]:
praat_script = """
procedure getTextGrid: .wav$

    # The folder I want to save the TextGrids to is "stimulus_textgrids"
    .path_temp$ = replace$: .wav$, ".wav", ".TextGrid", 0
	.path$ = replace$: .path_temp$, "stimuli", "stimulus_textgrids", 0

	.basename_temp$ = replace$: .path$, "/Users/sidneyma/Desktop/school/lign199/stimulus_textgrids/", "", 0
	.basename$ = replace$: .basename_temp$, ".TextGrid", "", 0
    # Print the basename so I can copy and paste it as the annotation
	writeInfoLine: .basename$

    ...

endproc
"""

Using this script, I was able to annotate all 495 files in about one hour. Definitely tedious, but probably the most efficient way to collect all the TextGrids.

## Duration logging

Now that we have all the TextGrid files, we can automatically collect the time-to-targets by logging the durations of the annotations we made. I decided to do this using Python's `praatio` module:

In [3]:
import pandas as pd

import os
import praatio
from praatio import tgio

In [4]:
def get_duration(textgrid_path, annotation):
    
    tg = tgio.openTextgrid(textgrid_path)
    first_tier = tg.tierDict[tg.tierNameList[0]]
    
    for entry in first_tier.entryList:
        start_time, end_time, label = entry
        if label == annotation:
            return round(end_time - start_time, 3)

Let's find the durations of each file, and save this information into a csv-convertible txt file:

In [5]:
dir_name = '/Users/sidneyma/Desktop/school/lign199'

# Name this file to 'durations.txt'
text_file_path = os.path.join(dir_name, 'durations.txt')
    
with open(text_file_path, 'w') as file:
    file.write('file, time_to_target\n')
    
dir_name = os.path.join(dir_name, 'stimulus_textgrids')

for file in os.listdir(dir_name):
    if file.endswith('.TextGrid'):
        file_path = os.path.join(dir_name, file)
        annotation = file.replace('.TextGrid', '')
        duration = get_duration(file_path, annotation)
        
        with open(text_file_path, 'a') as txt_file:
            txt_file.write(f"{annotation}, {duration}\n")

In [6]:
f = open(text_file_path, 'r')
print(f.read()[:300], '...')

file, time_to_target
at_nodish_oppname_mc, 1.755
bl_nocake_cm, 2.5
at_nocake_mg, 1.63
SC_nowax_mc, 1.917
SC_nowish_cm, 2.078
sc_nocake_mm, 1.836
SC_mypod_mg, 2.089
bl_iheat_cc, 2.219
SC_mysmoke_cm, 2.157
at_mypog_cm, 1.81
SC_myyoke_cm, 1.897
at_nocatch_mm, 1.84
sc_mypog_oppname_cg, 2.123
at_noweight ...


Perfect - now we have all the durations! Once necessary, we'll just convert `durations.txt` to a csv file.